- 基本概念
    - tf.logging
    - tf.summary
    - tf.Print
    - tf.estimator
    - tf.data
    - tf.nn
    - TFRecord
    - tf中的Feature
    - sess
    - 数据类型 
    - 激活函数
    - 损失函数
    - 优化器
    - 指标
    - 异常
- 变量
- 常见张量计算
- 常见张量操作
- 基本流程
- 构建网络
    - scope
    - 嵌入层及其他网络层
    - 其他操作: 归一化
- Keras

In [ ]:
## tensorflow.compat.v1 as tf基础
- tf.app.run(main=None, argv=None): 处理flag解析，然后执行main函数
- tf.compat.v1.disable_v2_behavior(): This function can be called at the beginning of the program(before Tensors, Graphs or other structures have been created, and before devices have been initialized. It switches all global behaviors that are different between TensorFlow 1.x and 2.x to behave as intended for 1.x.
- tf.gfile.Exists(filename)
- tf.compat.v1.Session(target='', graph=None, config=None): 运行tensorflow操作的类
    - tf.Session.reset(FLAGS.master)
    - sess = tf.Session(FLAGS.master)
    - init = tf.global_variables_initializer()
    - sess.run(init)
    - 开始迭代，每次迭代执行sess.run
- tf.keras.Model: 网络需要继承的类
    - 两个函数: build和call
- tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(labels, logits, axis=None, name=None, dim=None)

## tensorflow_probability as tfp基础
- tfp.distributions.OneHotCategorical(logits=None, probs=None, dtype=tf.int32): 由一组类的对数概率参数化；参数logits或probs指明最可能的类

## 基本概念

In [1]:
import tensorflow as tf
# tensorflow1.0: 静态图
tf.__version__

'1.15.0'

In [2]:
# tf.logging 本身就调用了 logging ，会产生两套 handlers，使得每次tf.logging.info("...") 时会将其中内容打印两遍
tf.logging.info("...")

INFO:tensorflow:...


In [ ]:
# tf.compat.v1.logging
    # set_verbosity(tf.compat.v1.logging.INFO)
    # 除了INFO(20)，还有DEBUG(10)，WARN(30)，ERROR(40)和FATAL(50)

In [ ]:
# tf.summary: 能够保存训练过程以及参数分布图并在tensorboard显示
    # tf.summary.scalar(tags, values, collections=None, name=None): 用来显示标量信息
    # merged = tf.summary.merge_all()
    # train_writer = tf.summary.FileWriter(tflog_path, sess.graph)
    # train_writer.add_summary(summary, train_iter)

In [ ]:
# tf.Print(input, data, message=None, summarize=None, name=None)
    # 最低要求两个输入，input和data，input是需要打印的变量的名字，data要求是一个list，里面包含要打印的内容
    # message是需要输出的错误信息
    # summarize是对每个tensor只打印的条目数量，如果是None，对于每个输入tensor只打印3个元素
    # name是op的名字

In [10]:
x=tf.constant([2,3,4,5])
x=tf.Print(x,[x,x.shape,'test', x],message='Debug message:',summarize=2)
with tf.Session() as sess:
    print(x.eval())

[2 3 4 5]


In [ ]:
# tf.estimator
    # 如果使用Estimator，就不用再使用Session
    # tf.estimator.RunConfig: 用于控制内部和checkpoints等
        # model_dir: 指定存储模型参数，graph等的路径
        # save_summary_steps: 每隔多少step就存一次Summaries
        # save_checkpoints_steps:每隔多少个step就存一次checkpoint

        # save_checkpoints_secs: 每隔多少秒就存一次checkpoint，不可以和save_checkpoints_steps同时指定。如果二者都不指定，则使用默认值，即每600秒存一次。如果二者都设置为None，则不存checkpoints
        # keep_checkpoint_max：指定最多保留多少个checkpoints，也就是说当超出指定数量后会将旧的checkpoint删除。当设置为None或0时，则保留所有checkpoints
        # log_step_count_steps:该参数的作用是,(相对于总的step数而言)指定每隔多少step就记录一次训练过程中loss的值，同时也会记录global steps/s，通过这个也可以得到模型训练的速度快慢
    # tf.estimator.Estimator(model_fn, config, params, model_dir)，
        # model_fn为自定义的网络模型函数
        # config用于控制内部和checkpoints等
        # params该参数的值会传递给model_fn
        
        # model_dir指定checkpoints和其他日志存放的路径
        # estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
        # estimator.evaluate(input_fn=test_input_fn, steps=None)
        # estimator.predict(predict_input_fn)
        # 定义在miniconda3/envs/ctrtf/lib/python3.6/site-packages/tensorflow_estimator/python/estimator下的estimator.py文件中
    # tf.estimator.experimental.stop_if_no_decrease_hook:
        # 定义early_stopping
        # 参数:
            # estimator: 模型
            # metric_name: 需要追踪的评估指标名称
            # max_steps_without_increase: 指定评估指标没有提升的情况下最大运行步数
    # tf.estimator.TrainSpec(input_fn, max_steps=None, hooks=None)
        # input_fn: 用来指定数据输入
        # max_steps: 用来指定训练的最大步数
        # hooks: 用来在 session 运行的时候做一些额外的操作
        # 定义在miniconda3/envs/ctrtf/lib/python3.6/site-packages/tensorflow_estimator/python/estimator下的training.py文件中
    # tf.estimator.EvalSpec(input_fn, steps, throttle_secs)
        # steps: 用来指定评估的迭代步数，如果为None，则在整个数据集上评估
        # throttle_secs: 多少秒后又开始评估，如果没有新的 checkpoints 产生，则不评估，所以这个间隔是最小值
        # 定义在miniconda3/envs/ctrtf/lib/python3.6/site-packages/tensorflow_estimator/python/estimator下的training.py文件中
    # tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
        # 定义在miniconda3/envs/ctrtf/lib/python3.6/site-packages/tensorflow_estimator/python/estimator下的training.py文件中
    # tf.estimator.EstimatorSpec
        # mode: ModeKeys
        # predictions
        # loss: 训练损失Tensor
        # train_op
        # eval_metric_ops
        # 不同模式需要传入不同参数，对于mode == ModeKeys.TRAIN:必填字段是loss和train_op；对于mode == ModeKeys.EVAL:必填字段是loss；对于mode == ModeKeys.PREDICT:必填字段是predictions
    # tf.estimator.ModeKeys.TRAIN
    # tf.estimator.ModeKeys.EVAL
    # tf.estimator.ModeKeys.PREDICT

In [ ]:
# tf.data
    # tf.data.TextLineDataset
        # 可处理csv和libsvm文件
    # tf.data.TFRecordDataset
        # 可处理tfrecord文件

In [ ]:
# tf.nn
    # tf.nn.bias_add
    # logits = tf.nn.bias_add(logits, output_bias)
    # log_probs = tf.nn.log_softmax(logits, axis=-1)

In [ ]:
# tf.python_io.TFRecordWriter(path)-->tf.io.TFRecordWriter
    # 为什么使用TFRecord?: TFRecord格式的文件存储形式会很合理的帮我们存储数据。TFRecord内部使用了“Protocol Buffer”二进制数据编码方案，它只占用一个内存块，只需要一次性加载一个二进制文件的方式即可，简单，快速，尤其对大型训练数据很友好。而且当我们的训练数据量比较大的时候，可以将数据分成多个TFRecord文件，来提高处理效率
    # writer.write(example.SerializeToString())，其中example.SerializeToString()是将example中的map压缩为二进制文件，更好的节省空间
    # example的构建: tf.train.Example(features = None)+tf.train.Features(feature = None)，其中feature是个map，key是要保存数据的名字，value是要保存的数据，格式必须符合tf.train.Feature实例要求
    # writer.close()

In [ ]:
# tf.FixedLenFeature和tf.VarLenFeature-->tf.io.FixedLenFeature和tf.io.VarLenFeature
    # 前者返回一个定长的tensor，后者返回不定长的；前者可传入shape参数，如为[]则要求tensor的shape=(batch_size,)，若为[k]，则输出tensor的shape=(batch_size,k)

In [ ]:
# tf.feature_column和tfrecord搭配使用，特征名要一致
# tf.feature_column.categorical_column_with_identity(): 
    # 参数有特征名、num_buckets和default_value
    # 得到one-hot编码，即输出为N*num_buckets的矩阵
# tf.feature_column.embedding_column
    # 参数有categorical_column、dimension
    # 将sparse/categrical特征转化为dense向量
# tf.feature_column.numeric_column
    # 参数有特征名
    # 用于抽取数值类型的特征，即dense特征
# tf.feature_column.input_layer(features, feature_columns)
    # config = tf.ConfigProto();session = tf.Session(config=config)

In [ ]:
# sess
# sess.run()这个里面只能放operation和tensor，还有参数feed_dict
# tf.ConfigProto()主要的作用是配置tf.Session的运算方式，比如gpu运算或者cpu运算
    # 参数如device_count={'GPU':0, 'CPU':4}
# _, loss, cos_loss, output, summary = sess.run([train_op, loss, cos_loss, img_last_layer, merged], feed_dict={img: images, img_label: labels})
# sess.close()

import keras.backend.tensorflow_backend as KTF
# config = tf.ConfigProto()  
# config.gpu_options.allow_growth=True / config.allow_soft_placement = True 
# session = tf.Session(config=config)
# KTF.set_session(session)

In [7]:
x = tf.constant(1)
sess = tf.InteractiveSession()
print(x.eval())

1


In [ ]:
# dtype
    # tf.int64
    # tf.float32
    # tf.bool
# tf.to_float
# tf.to_int32
# tf.cast

In [ ]:
# 激活函数
    # tf.nn.softmax
    # tf.nn.sigmoid
    # tf.nn.relu
    # tf.nn.sotfplus

    # tf.sigmoid

In [ ]:
# 损失函数
    # tf.nn.sigmoid_cross_entropy_with_logits() # logits的输出维度可为1
    # tf.nn.softmax_cross_entropy_with_logits() # 其中labels要求是onehot，logits的输出维度为类别数

# tf.keras.backend.binary_crossentropy

In [ ]:
# 优化器
    # tf.train.AdagradOptimizer

In [ ]:
# 指标
    # tf.metrics.accuracy
        # accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
    # tf.metrics.auc
    # tf.metrics.recall
    # tf.metrics.precision
    # tf.metrics.true_positives
    # tf.metrics.true_negatives
    # tf.metrics.false_positives
    # tf.metrics.false_negatives
    # tf.contrib.metrics.f1_score

In [ ]:
# 异常
    # InvalidArgumentError: 当操作接收到无效参数时触发.例如,如果某个操作接收到的输入张量具有无效的值或形状,则可能发生该情况.

## 变量

In [ ]:
# tf.constant(list) | tf.constant(list, shape=[2, 3])

In [4]:
a = tf.constant([1,2,3])
print(a)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a))

Tensor("Const_2:0", shape=(3,), dtype=int32)
[1 2 3]


In [ ]:
# tf.range()

In [ ]:
# tf.identity(input_tensor)：返回一个和input_tensor形状、内容都相同的tensor

In [5]:
b = tf.identity(a)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(b))

[1 2 3]


In [ ]:
# tf.ones_like | tf.zeros_like
# tf.zeros() # 其中shape参数可用tf.shape指定

In [6]:
# tf.one_hot(indices, depth, on_value=None, off_value=None) # indices给出一个索引的列表，depth表示类别数，即最后onehot向量的长度
    # on_value默认值为1，off_value默认值为0
    # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
# tf.nn.top_k(input, k) # 这个函数表示要返回input中每行元素最大的值，并返回它们的索引下标
a = tf.constant([[1,2,3],[4,4,6]])
v, i = tf.nn.top_k(a, 2)
values = tf.reshape(v, [-1])
output_shape = tf.stack([2,3])
idx = tf.constant([[j]*2 for j in range(2)])
idx_u = tf.expand_dims(idx, 2)
i_u = tf.expand_dims(i, 2)
indices = tf.concat([idx_u, i_u], 2)
indices = tf.reshape(indices, [-1, indices.get_shape().as_list()[-1]])
# c = tf.sparse_to_dense(indices, output_shape, values) # 仍有问题，因为indices的必须按顺序\
indices_sort = tf.sort(i, 1)
indices_sort_u = tf.expand_dims(indices_sort, 2)
indices_b = tf.concat([idx_u, indices_sort_u], 2)
indices_b = tf.reshape(indices_b, [-1, indices_b.get_shape().as_list()[-1]])
c = tf.sparse_to_dense(indices_b, output_shape, tf.constant(1))
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(v))
    print(sess.run(i))
    # print(sess.run(idx))
    # print(sess.run(values))
    # print(sess.run(indices))
    # print(sess.run(c))
    print(sess.run(indices_sort))
    print(sess.run(indices_b))
    print(sess.run(c))
    print(sess.run(a*c))
    # print(sess.run(tf.ones([2,3]) * (-2 ** 32 + 1)))
    print(sess.run(tf.where(tf.cast(c, tf.bool), tf.constant([[9,9,9],[9,9,9]]), tf.constant([[8,8,8],[8,8,8]]))))

[[1 2 3]
 [4 4 6]]
[[3 2]
 [6 4]]
[[2 1]
 [2 0]]
[[1 2]
 [0 2]]
[[0 1]
 [0 2]
 [1 0]
 [1 2]]
[[0 1 1]
 [1 0 1]]
[[0 2 3]
 [4 0 6]]
[[8 9 9]
 [9 8 9]]


In [21]:
classes = 3
labels = tf.constant([0,1,2])
output = tf.one_hot(labels, classes)
with tf.Session() as sess:
    print(sess.run(output))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
# tf.random_normal(shape=[2,3], mean=0, stddev=1) # 默认均值为0，标准差为1
    # tf.random.truncated_normal() # 参数同上；若生成的数在某个区间之外，则重新进行生成，保证生成值在均值附近
    # shape参数可由tf.shape提供
# tf.random_uniform(shape, minval, maxval)

In [ ]:
# 不能直接将tf.Tensor作为bool

In [20]:
# tf.SparseTensor(values,indices,dense_shape)：三个参数均可使用np.array定义，定义稀疏tensor，可以节省内存
# tf.sparse_tensor_to_dense(sp_input,default_value=0)：将 SparseTensor 转换为稠密张量
# tf.sparse_to_dense(sparse_indices, output_shape, sparse_values, default_value=0)
sparse_indices = tf.constant([[0,3],[2,3]])
output_shape = tf.stack([3,4])
sparse_values = tf.constant([22,33])
a = tf.sparse_to_dense(sparse_indices, output_shape, sparse_values)
print(output_shape)
with tf.Session() as sess:
    print(output_shape[0])
    print(sess.run(a))

Tensor("stack_3:0", shape=(2,), dtype=int32)
Tensor("strided_slice_2:0", shape=(), dtype=int32)
[[ 0  0  0 22]
 [ 0  0  0  0]
 [ 0  0  0 33]]


In [ ]:
# tf.Variable()和tf.get_variable()有不同的创建变量的方式：
    # tf.Variable() 每次都会新建变量。如果希望重用（共享）一些变量，就需要用到了get_variable()，它会去搜索变量名，有就直接用，没有再新建
# tf.Variable(initial_value=0, name="...", trainable=False, dtype=tf.int64, collections=[...])
# tf.get_variable(name="...", shape, collections=[...], initializer=tf.truncated_normal_initializer(stddev=0.001))
    # 其中参数shape的值不能为None
    # initializer还可取值：tf.zeros_initializer()
    # 用tf.float32指明数据类型没用，均是dtype=float32_ref

In [ ]:
# from tensorflow.contrib.framework.python.ops import variables as contrib_variables
# contrib_variables.model_variable('...', shape, initializer=tf.zeros_initializer(), trainable)

# from tensorflow.python.ops import nn_ops
# nn_ops.bias_add

In [ ]:
# tf.get_collection(key)：可以用来获取key集合中的所有变量，返回一个列表。如GraphKeys类包含许多标准集合名，如tf.GraphKeys.REGULARIZATION_LOSSES、tf.GraphKeys.VARIABLES、tf.GraphKeys.TRAINABLE_VARIABLES、tf.GraphKeys.GLOBAL_STEP、tf.GraphKeys.GLOBAL_VARIABLES、tf.GraphKeys.UPDATE_OPS

# tf.add_to_collection('all_loss', self.loss)
# tf.get_collection('all_loss')

In [ ]:
# ndarray转化为tensor
    # 通过 convert_to_tensor 这个函数进行转换

## 常见张量计算

In [ ]:
# tf.reduce_sum | tf.reduce_mean | tf.reduce_max
    # 如果不指定维度(axis)，则计算所有元素

In [5]:
# tf.unsorted_segment_sum(data, segment_ids, num_segments, name=None)
    # data为需要计算的tensor
    # segment_ids也是一个tensor，对应元素的值标志着data的下标
    # num_segments是一个 tensor,必须是以下类型之一：int32,int64，对应着最后生成结果的维度
    # 计算一个张量,使得 output[i] = data[index]，其中index为对应segment_ids中元素值为i的位置 
import numpy as np
data = tf.constant(np.arange(0,9).reshape(3,3))
segment_ids = tf.constant([2,0,1])
num_segments = 3
with tf.Session() as sess:
    print(sess.run(tf.unsorted_segment_sum(data, segment_ids, num_segments)))

[[3 4 5]
 [6 7 8]
 [0 1 2]]


In [9]:
b,k,n=3,3,4
indices=tf.constant([[1,2,3],[0,2,3],[0,1,3]])
values=tf.constant([[0,1,2],[3,4,5],[6,7,8]]) # 原始矩阵[[-1,0,1,2],[3,2,4,5],[6,7,2,8]]
indices_flat = tf.reshape(indices, [-1]) + tf.div(tf.range(b * k), k) * n
ret_flat = tf.unsorted_segment_sum(tf.reshape(values, [-1]), indices_flat, b * n)
with tf.Session() as sess:
    print(sess.run(tf.div(tf.range(b * k), k) * n))
    print(sess.run(indices_flat))
    print(sess.run(ret_flat))
    print(sess.run(tf.reshape(ret_flat, [b, n])))

[0 0 0 4 4 4 8 8 8]
[ 1  2  3  4  6  7  8  9 11]
[0 0 1 2 3 0 4 5 6 7 0 8]
[[0 0 1 2]
 [3 0 4 5]
 [6 7 0 8]]


In [ ]:
# tf.pow | tf.square | tf.add | tf.sqrt | tf.exp
# tf.squared_difference(x,y): 计算张量 x、y 对应元素差平方

In [ ]:
# 乘法
    # tf.multiply(x, y)：两个矩阵中对应元素各自相乘
    # tf.matmul(a, b)：将矩阵a乘以矩阵b
        # logits = tf.matmul(output_layer, output_weights, transpose_b=True) 其中output_layer最后的维度是hidden_size，output_weights的shape是[num_labels, hidden_size]
    # tf.tensordot(a, b, axes)
        # 在axes=2轴上相乘指的是将a的最后两个维度与b的前两个维度矩阵相乘，然后将结果累加求和，消除(收缩)这四个维度，矩阵a，b剩下的维度concat，就是所求矩阵维度
        # 在axes=[[1,3],[0,2]]上进行tensor相乘，指的是将a的第一个维度、第三个维度concat的维度与b的第0(维度下标从0开始)个维度、第二个维度concat的维度进行矩阵相乘，然后将结果累加求和，消除(收缩)这四个维度，矩阵a，b剩下的维度concat，就是所求矩阵维度

In [2]:
a = tf.constant([0,1,2,1,3,4,5,2,3,4,5,0],shape=[2,1,3,2])
b =tf.constant([1,3,2,3,1,2],shape=[2,3,1])
res1 = tf.tensordot(a,b,axes=2)
res2 = tf.tensordot(a,b,axes=[[1,3],[0,2]])
print("a的shape:",a.shape)
print("b的shape:",b.shape)
print("res1 shape:",res1.shape)
print("res2 shape:",res2.shape)

a的shape: (2, 1, 3, 2)
b的shape: (2, 3, 1)
res1 shape: (2, 1, 1)
res2 shape: (2, 3, 3)


In [ ]:
# tf.logical_or(x, y)：期望输入布尔类型。输入类型为张量，如果张量包含多个元素，则将按元素进行逻辑或运算

In [8]:
# 计算均值和方差
img = tf.Variable(tf.random_normal([2, 3]))
mean, variance = tf.nn.moments(img, axes=0)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(img))
    print(sess.run([mean, variance]))

[[ 1.3102722  -0.9942122  -1.876304  ]
 [-0.16043976 -0.2800879   0.8441572 ]]
[array([ 0.57491624, -0.63715005, -0.5160734 ], dtype=float32), array([0.5407484 , 0.12749338, 1.8502275 ], dtype=float32)]


In [11]:
# tf.math.accumulate_n
a=tf.constant(-1)
b=tf.constant([1,2,3])
with tf.Session() as sess:
    print(sess.run(tf.math.accumulate_n([a,a,a])))
    print(sess.run(tf.math.accumulate_n([b,b,b])))

-3
[3 6 9]


In [ ]:
# tf.math.reduce_prod
# tf.math.reduce_sum
# tf.math.log

## 常见张量操作

In [ ]:
# tensor.get_shape().as_list()
    # 若获取的对应值是None，如batch_size，一般采用tf.shape(tensor)[]来获取(可在reshape函数中使用)；但在tf.compat.v1.get_variable中的shape参数不接受张量
# tf.shape：获取张量的形状；返回的是一个tensor
# tensorflow打印张量shape，如遇自定义变量(batch size)则会显示None
# tf.reshape(tensor, shape)：若shape为[]，输出标量；若shape为[-1]，则展平张量
# tf.expand_dims(input,axis=None)：在axis轴处给input增加一个维度

In [ ]:
# 该函数可以返回一个tensor的形状,即使我们的tensor定义时某一维的形状定义为None
# def get_shape(tensor):
#     static_shape = tensor.shape.as_list()
#     dynamic_shape = tf.unstack(tf.shape(tensor)) # 其实加不加tf.unstack都行，结果都是tensor
#     dims = [s[1] if s[0] is None else s[0]
#             for s in zip(static_shape, dynamic_shape)]
#     return dims

In [2]:
a = tf.random_normal(shape=[2,3])
b = a[:, 2]
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(tf.expand_dims(b, 1)))

[[-0.01855624  0.9438359  -0.8169631 ]
 [ 0.5846827  -0.23804206 -1.5526854 ]]
[[-0.15797293]
 [-1.8465679 ]]


In [8]:
# tf.concat(values, axis)：按照指定的已经存在的轴进行拼接
# tf.stack(values, axis)：按照指定的新建的轴进行拼接
    # tf.stack() 替代 tf.pack()
# tf.unstack(num=None, axis=0)：矩阵分解

In [ ]:
# tf.tile(input, multiples)：两个参数形状需要一致，第二个参数指明每个维度的复制次数

In [ ]:
# tf.split(input_tensor,num_or_size_splits,axis=0)：num_or_size_splits如果是个整数n，就将输入的tensor分为n个子tensor。如果是个list T，就将输入的tensor分为len(T)个子tensor

In [6]:
c = tf.random_normal([2,3])
cs = tf.split(c, 2, axis=0)
print(type(cs))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(cs))

<class 'list'>
[array([[-0.3797491 ,  0.39872998, -0.23381083]], dtype=float32), array([[-1.3213638 , -0.13312116, -1.4646379 ]], dtype=float32)]


In [ ]:
# tf.argmax | tf.maximum | tf.squeeze
	# tf.argmax(log_probs, axis=-1, output_type=tf.int32)

In [ ]:
# tf.equal(A, B)：对比这两个矩阵或者向量的相等的元素，如果是相等的那就返回True，否则返回False，返回的值的矩阵维度和A是一样的
# tf.greater(x, y)
# tf.less(x, y)
# tf.not_equal(x, y)

In [ ]:
# tf.where(condition, x=None, y=None)：
# 若只有参数condition，则返回condition中为True的索引
# 若有x和y，则x, y相同维度，返回值是对应元素，condition中元素为True的元素替换为x中的元素，为False的元素替换为y中对应元素

In [ ]:
# tf.sequence_mask(lengths,maxlen=None,dtype=tf.bool)：用于数据填充。返回值mask张量：默认mask张量就是布尔格式的一种张量表达，只有True和 False 格式，也可以通过参数dtype指定其他数据格式。参数lengths：表示的是长度；可以是标量，也可以是列表 [ ] ，也可以是二维列表[ [ ],[ ] ,…]，甚至是多维列表…，一般列表类型的用的比较多。参数maxlen：当默认None，默认从lengths中获取最大的那个数字，决定返回mask张量的长度；当为N时，返回的是N长度

In [5]:
tf.sequence_mask(tf.constant([2,3,4]), 5)

<tf.Tensor 'SequenceMask/Less:0' shape=(3, 5) dtype=bool>

In [ ]:
# tf.assign(A, new_number)：这个函数的功能主要是把A的值变为new_number
# tf.assign_add(ref,value)：更新ref的值，通过增加value，即：ref = ref + value

In [ ]:
# tf.cond()类似于if...else...

In [2]:
# tf.gather()用来取出tensor中指定索引位置的元素
temp1 = tf.range(0,10)*10
temp2 = tf.constant(1, shape=[10])
temp3 = temp1+temp2
temp4 = tf.gather(temp3, [1,5,9])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(temp1))
    print(sess.run(temp2))
    print(sess.run(temp3))
    print(sess.run(temp4))

[ 0 10 20 30 40 50 60 70 80 90]
[1 1 1 1 1 1 1 1 1 1]
[ 1 11 21 31 41 51 61 71 81 91]
[11 51 91]


In [12]:
# tf.scatter_nd对零张量进行赋值
# updates的形状：indices.shape[:−1]+shape[indices.shape[−1]:]=(4,)+(,)=(4,)
indices = tf.constant([[4], [3], [1], [7]])
updates = tf.constant([9, 10, 11, 12])
shape = tf.constant([8])
scatter = tf.scatter_nd(indices, updates, shape)
print(scatter)
scatter = tf.cast(scatter, tf.float32)
print(scatter)
with tf.Session() as sess:
    print(sess.run(scatter))

Tensor("ScatterNd_5:0", shape=(8,), dtype=int32)
Tensor("Cast_1:0", shape=(8,), dtype=float32)
[ 0. 11.  0. 10.  9.  0.  0. 12.]


In [2]:
# tf.slice 抽取数据
    # begin表示从inputs的哪几个维度上的哪个元素开始抽取
    # size表示在inputs的各个维度上抽取的元素个数
    # 若begin[]或size[]中出现-1,表示抽取对应维度上的所有元素
x = [[1,2,3],[4,5,6]]
with tf.Session() as sess:
    begin = [0,1] # 从x[0,1]，即元素2开始抽取
    size = [2,1] # 从x[0,1]开始，对x的第一个维度(行)抽取2个元素，对x的第二个维度(行)抽取1个元素
    print(sess.run(tf.slice(x, begin, size)))

[[2]
 [5]]


In [ ]:
# tf.control_dependencies()函数是用来控制计算流图的，也就是给图中的某些计算指定顺序
    # 有的时候我们想要指定某些操作执行的依赖关系，比如想要让参数先更新，然后再获取参数更新后的值等
    # 如tf.control_dependencies([a,b]): 按先后顺序执行a、b种操作
# tf.assign_add

## 基本流程

In [ ]:
# b = tf.Variable(tf.zeros([100])) # 100维向量
# W = tf.Variable(tf.random_uniform([784, 100], -1, 1))
# x = tf.placeholder(dtype=tf.float32, name="x")  / img = tf.placeholder(tf.float32, [None, 256, 256, 3])
# relu = tf.nn.relu(tf.matmul(W, x)+b)
# C = [...] # Cost
# s = tf.Session()
# for step in range(0, 10):
#     input = 100维向量
#     result = s.run(C, feed_dict={x:input})
#     print(step, result)

## 构建网络

In [ ]:
# TensorFlow变量作用域
    # tf.name_scope()、tf.variable_scope()是两个作用域函数，一般与两个创建/调用变量的函数tf.Variable() 和tf.get_variable()搭配使用。常用于：1）变量共享；2）tensorboard画流程图进行可视化封装变量
    # tf.name_scope()/命名域、tf.variable_scope()/变量域会在模型中开辟各自的空间，而其中的变量均在这个空间内进行管理
        # 命名域还可由tf.op_scope创建，变量域还可由tf.variable_op_scope创建
        # 对于上述两种作用域，对于使用tf.Variable()方式创建的变量，具有相同的效果，都会在变量名称前面，加上域名称。对于通过tf.get_variable()方式创建的变量，只有variable scope名称会加到变量名称前面，而name scope不会作为前缀
    # variable_scope可以通过设置reuse标志以及初始化方式来影响域下的变量，因为想要达到变量共享的效果, 就要在 tf.variable_scope()的作用域下使用 tf.get_variable() 这种方式产生和提取变量. 不像 tf.Variable() 每次都会产生新的变量, tf.get_variable() 如果遇到了已经存在名字的变量时, 它会单纯的提取这个同样名字的变量，如果不存在名字的变量再创建
    # 必须要在tf.variable_scope的作用域下使用tf.get_variable()函数。这里用tf.get_variable( ) 而不用tf.Variable( )，是因为前者拥有一个变量检查机制，会检测已经存在的变量是否设置为共享变量，如果已经存在的变量没有设置为共享变量，TensorFlow 运行到第二个拥有相同名字的变量的时候，就会报错
    # with tf.variable_scope(name_or_scope="...", reuse=tf.AUTO_REUSE) as scope:
    # with tf.name_scope("..."):

In [3]:
with tf.name_scope("my_name_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32) 
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)
    print(v1.name)
    print(v2.name) 
    print(a.name)

var1:0
my_name_scope/var2:0
my_name_scope/Add:0


In [4]:
with tf.variable_scope("my_variable_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)
    print(v1.name) 
    print(v2.name)
    print(a.name)

my_variable_scope/var1:0
my_variable_scope/var2:0
my_variable_scope/Add:0


In [ ]:
from tensorflow.python.ops import variable_scope
# with variable_scope.variable_scope("name",reuse=tf.AUTO_REUSE):

In [ ]:
from tensorflow.contrib.framework.python.ops import arg_scope
# with arg_scope(): 指定网络的默认参数；第一个参数如[layers.conv2d]表示要执行操作的网络，后续的参数就是要设定默认的参数

In [ ]:
# 嵌入
# tf.nn.embedding_lookup(嵌入矩阵, 索引列表) 

In [ ]:
# from tensorflow.contrib import layers

In [ ]:
# layers.input_from_feature_columns(columns_to_tensors,feature_columns,scope=tf.variable_scope)
# layers.fully_connected(输入, 输出维度, activation_fn=tf.nn.softmax, variables_collections, outputs_collections, scope)
# layers.linear(输入, 输出维度, scope, variables_collections, outputs_collections)
# layers.batch_norm(input_vec, scale=True, is_training, scope, variables_collections, outputs_collections)

In [ ]:
# tf.layers.dense
# tf.layers.batch_normalization
# tf.layers.dropout

# output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

In [ ]:
# 归一化
# tf.nn.l2_normalize(张量)：元素除以各自的范数

### Keras

In [ ]:
# from tensorflow.python.keras import backend
# from tensorflow.python.keras.initializers import RandomNormal, Zeros, glorot_normal
# from tensorflow.python.keras.layers import Input, Lambda, Embedding, Layer, Flatten, Dropout, Dense
    # Input中参数shape不包括batch size，例如shape=(32,) 表示了预期的输入将是一批32维的向量
        # Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    # Layer
        # bulid()
        # call()
    # Embedding(len(word_dict), 300, trainable=True)
    # Flatten()(tensor)
    # Activation('softmax')(tensor)
    # Dot((1, 1))(tensor)
    # TimeDistributed，如原始形状为(32, 10, 16)，输出后为(32, 10, 8)，32为batch_size，10为时间步
# from tensorflow.python.keras.regularizers import l2

In [ ]:
# tf.keras.layers.Layer
# tf.keras.layers.Concatenate
# tf.keras.layers.add
# tf.keras.layers.Dense 第一个参数是输出维度
# tf.keras.models.Model(inputs, outputs)
    # 首先定义好网络，再将网络的输入和输出部分作为参数定义一个Model类对象
    # model.compile()
        # model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['acc'])
    # model.fit()
        # verbose: 日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为每个epoch输出一行记录
        # model.fit_generator(train_data_batch, epochs=1, steps_per_epoch=len(train_label)//30)
    # model.predict()
# tf.keras.initializers.glorot_normal
# tf.keras.initializers.RandomUniform(-gamma / 100, gamma / 100)

In [ ]:
# 命名空间
	# 遇到问题: UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb6 in position 2437: invalid start byte
	# 原因在于名字重复，发生碰撞；尽量为新建变量以及网络层(尤其涉及dense层的网络)赋予特定的名字

In [ ]:
# import keras
	# from keras.layers import * --> class Attention(Layer):  需要有build(self, input_shape)函数，定义相关权重，如self.WQ = self.add_weight(name='WQ', shape=(input_shape[0][-1], self.output_dim), initializer='glorot_uniform', trainable=True)；还需要有call(self, x)函数
	# from keras.models import Model
	# from keras import backend as K
		# mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
		# mask = 1 - K.cumsum(mask, 1) # 沿着张量的某一维累积和
		# K.expand_dims(mask, 2)
		# K.dot(Q_seq, self.WQ)
		# K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
		# K.permute_dimensions(Q_seq, (0,2,1,3))
		# K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
		# K.softmax()

In [ ]:
# import keras.backend.tensorflow_backend as KTF

In [ ]:
# from keras.utils import Sequence：定义generator

In [ ]:
dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)

In [ ]:
tf.gfile.DeleteRecursively(OUTPUT_DIR)
tf.gfile.MakeDirs(OUTPUT_DIR)
with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:

In [ ]:
import tensorflow_hub as hub
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
with tf.Graph().as_default():
	bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])


In [ ]:
import tensorflow_hub as hub
bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
bert_inputs = dict(
input_ids=input_ids,
input_mask=input_mask,
segment_ids=segment_ids)
bert_outputs = bert_module(
inputs=bert_inputs,
signature="tokens",
as_dict=True)

In [ ]:
import bert
tokenizer = bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

bert.run_classifier.InputExample
bert.run_classifier.convert_examples_to_features
bert.run_classifier.input_fn_builder

In [ ]:
tf.placeholder_with_default

In [ ]:
tf.unstack 对矩阵进行分解的函数
# value：代表需要分解的矩阵变量
# axis：指明对矩阵的哪个维度进行分解
# num
tf.stack 对矩阵进行拼接

In [ ]:
tf.random_crop # 随机裁剪函数
tf.image.random_flip_left_right # 随机左右翻转图片
tf.image.flip_left_right
tf.image.crop_to_bounding_box

In [ ]:
tf.nn.conv2d(input, filter, strides, padding)
# input: 形如[batch, in_height, in_width, in_channels]
# filter: 卷积核 形如[filter_height, filter_width, in_channels, out_channels]
# strides: 卷积时在图像每一维的步长，这是一个一维的向量，长度4
# padding: string类型的量，只能是"SAME","VALID"其中之一，这个值决定了不同的卷积方式

In [ ]:
tf.nn.max_pool(value, ksize, strides, padding, name=None)
# value: 形如[batch, height, width, channels]
# ksize: 池化窗口的大小，取一个四维向量，一般是[1, height, width, 1]，因为我们不想在batch和channels上做池化，所以这两个维度设为了1
# strides: 和卷积类似，窗口在每一个维度上滑动的步长，一般也是[1, stride,stride, 1]
# padding: 和卷积类似，可以取'VALID' 或者'SAME'
# 返回形如[batch, height, width, channels]

In [ ]:
tf.nn.local_response_normalization(input, depth_radius=None, bias=None, alpha=None, beta=None, name=None)
# 正则
# input: 形如[batch, height, width, channels]

In [ ]:
shape = int(np.prod(pool5.get_shape()[1:]))
pool5_flat = tf.reshape(pool5, [-1, shape])

tf.reshape(tf.reduce_sum(t, 1), [tf.shape(t)[0], 1])

In [ ]:
tf.transpose
tf.clip_by_value

In [ ]:
tf.abs
tf.boolean_mask
tf.subtract

In [ ]:
tf.train.exponential_decay
# 指数衰减法设置学习率
# global_step=tf.Variable(0)
# decay_rate为衰减系数
# decay_steps为衰减速度
# staircase(默认值为False)选择不同的衰减方式：如果staircase = True，那就表明每decay_steps次计算学习速率变化，更新原始学习速率，如果是False，那就是每一步都更新学习速率

In [ ]:
tf.train.GradientDescentOptimizer(lr).minimize(....., global_step=global_step)

opt = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)
grads_and_vars = opt.compute_gradients(loss, train_layers) # train_layers为网络层参数列表
opt.apply_gradients([(grads_and_vars[0][0], train_layers[0]), ...], global_step=global_step)